### Import some required libraray

In [1]:
import string
import traceback
import re
import os
import json
import pickle
import glob
from PIL import Image
from time import time

from sklearn.model_selection import train_test_split### 1. Best company in a sector based on ROCE
from sklearn.preprocessing import MinMaxScaler

import cv2
import numpy as np
from numpy import array
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
import tensorflow as tf
import tensorflow.compat.v1
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils
from keras.layers.embeddings import Embedding
from keras.layers.merge import add
from keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Attention, Layer,
                                     Add,concatenate,
                                     Input, 
                                     Dense,  
                                     LSTM, Bidirectional, GRU,
                                     ZeroPadding2D, 
                                     Convolution2D, Conv2D, 
                                     GlobalAveragePooling2D, GlobalAvgPool2D, GlobalMaxPooling2D, GlobalMaxPool2D, 
                                     AveragePooling2D, AvgPool2D, MaxPooling2D, MaxPool2D,
                                     Flatten,
                                     BatchNormalization, 
                                     Dropout)

from tensorflow.keras.layers import (Activation, 
                                     ReLU, 
                                     LeakyReLU, 
                                     Softmax)

from tensorflow.keras.optimizers import (SGD,
                                         Adam,
                                         Adagrad,
                                         Adadelta,
                                         RMSprop,
                                         Nadam)

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

# from tensorflow.keras.applications.vgg16 import VGG16

from tqdm import tqdm
from tensorflow.keras.applications.vgg16 import VGG16
import pyphen
import eng_to_ipa as ipa  # international phonetic aphabet

### Load the dataset

In [2]:
df = pd.read_csv('stock_data.csv', usecols=['id', 'ROCE(%)', 'Income Growth(%)','Sale Growth(%)', 'Debt Equity Ratio'])
df

,id,ROCE(%),Income Growth(%),Sale Growth(%),Debt Equity Ratio
0,1,105.502993,137.380475,-38.862170,0.493548
1,2,-290.386986,91.287783,-92.688491,-0.667857
2,3,-34.064807,-20.420386,16.818664,0.325987
3,4,9.708701,67.468809,-43.614209,0.432126
4,5,123.381079,-97.451894,29.146933,0.297247


### 1. Best company in a sector based on ROCE

ROCE is a metric for analyzing profitability and for comparing profitability levels across companies in terms of capital. Two components are required to calculate return on capital employed: earnings before interest and tax (EBIT) and capital employed.



In [3]:
df.loc[df['ROCE(%)'] == df['ROCE(%)'].max()]

,id,ROCE(%),Income Growth(%),Sale Growth(%),Debt Equity Ratio
6110,6111,371.215862,-107.696446,-125.083064,0.107304


IN our dataset the company whach id is 6111 has the highest value of ROCE, means this is the best company on the basis of ROCE(%)

### 2. Best company in a sector based on Income Growth(%)

In [4]:
df.loc[df['Income Growth(%)'] == df['Income Growth(%)'].max()]

,id,ROCE(%),Income Growth(%),Sale Growth(%),Debt Equity Ratio
2825,2826,51.354878,328.682196,8.440808,0.1922


IN our dataset the company whach id is 2826 has the highest value of Income growth, means this is the best company on the basis of Income Growth(%)

### 3. Best company in a sector based on Sale Growth(%)

In [5]:
df.loc[df['Sale Growth(%)'] == df['Sale Growth(%)'].max()]

,id,ROCE(%),Income Growth(%),Sale Growth(%),Debt Equity Ratio
4163,4164,-40.87505,-138.832515,362.967583,-0.347628


IN our dataset the company whach id is 4164 has the highest value of Sale Growth(%), means this is the best company on the basis of Sale Growth(%)

### 4. Best company in a sector based on Debt Equity Ratio

In [6]:
df.loc[df['Debt Equity Ratio'] == df['Debt Equity Ratio'].max()]

,id,ROCE(%),Income Growth(%),Sale Growth(%),Debt Equity Ratio
118,119,166.2318,45.854283,28.219699,0.499634


IN our dataset the company whach id is 119 has the highest value of Sale Growth(%), means this is the best company on the basis of Sale Growth(%)

### 5. Best company in a sector based on ROCE(%), Income Growth(%), Sale Growth(%), Debt Equity Ratio

#### 5.1 Normalize the dataset

Min Max converts every value of a column into a number between 0 and 1.

In [7]:
scaller = MinMaxScaler() 
np.array(df)
scalled_data = scaller.fit_transform(df.iloc[:, 1:])
scalled_data = pd.DataFrame(scalled_data, columns = df.columns[1:])
scalled_data['id'] = df['id']

In [8]:
scalled_data.iloc[:, 0:-1].max(), scalled_data.iloc[:, 0:-1].min()

(ROCE(%)              1.0
 Income Growth(%)     1.0
 Sale Growth(%)       1.0
 Debt Equity Ratio    1.0
 dtype: float64,
 ROCE(%)              0.0
 Income Growth(%)     0.0
 Sale Growth(%)       0.0
 Debt Equity Ratio    0.0
 dtype: float64)

After scalling we can see that every column has the maximum value of 1 and minimum value of 0


Best company should be that company which have high value in every field, so lets find that company which satisfy this condition. let's sum every value of each company. The best company should have the highest value

In [9]:
max_val = np.max(np.sum(scalled_data.iloc[:, 0:-1].values, axis=1))
output_record = scalled_data.loc[np.sum(scalled_data.iloc[:, 0:-1].values, axis=1) == max_val]
output_record

,ROCE(%),Income Growth(%),Sale Growth(%),Debt Equity Ratio,id
610,0.663473,0.900839,0.79237,0.945551,611


Now, We get the data of that company which is best company based on ROCE(%), Income Growth(%), Sale Growth(%), Debt Equity Ratio

But, This data is not the original data, We found the data in a scalled dataset to get the in a original format we have to fetch the record from the original dataframe which has the company id same as the scalled table output

In [10]:
df.loc[df['id'] == int(output_record['id'])]

,id,ROCE(%),Income Growth(%),Sale Growth(%),Debt Equity Ratio
610,611,133.39394,257.650889,191.964885,0.418008


This is the required result, here all company which id is 611 is the best company in a sector based on ROCE(%), Income Growth(%), Sale Growth(%), Debt Equity Ratio